In [3]:
import tarfile
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import torch, torchvision
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import matplotlib.pyplot as plt
from tempfile import TemporaryDirectory
from tqdm import tqdm
import os
from torchvision import datasets, models
import torch.nn as nn
from torch.optim import lr_scheduler
import time

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
with open("../data/train_lst.txt", "r") as f:
  train_paths = f.readlines()

train_paths = [curr_path[:-1] for curr_path in train_paths]

with open("../data/val_lst.txt", "r") as f:
  val_paths = f.readlines()

val_paths = [curr_path[:-1] for curr_path in val_paths]

In [9]:
dict_train = {
  "path": [],
  "label_idx": [],
  "label": []
}

dict_val = {
  "path": [],
  "label_idx": [],
  "label": []
}

for item in train_paths:
  curr_path, curr_label_idx, curr_label = item.split(" ")
  dict_train["path"].append("../data/" + curr_path)
  dict_train["label_idx"].append(curr_label_idx)
  dict_train["label"].append(curr_label)

for item in val_paths:
  curr_path, curr_label_idx, curr_label = item.split(" ")
  dict_val["path"].append("../data/" + curr_path)
  dict_val["label_idx"].append(curr_label_idx)
  dict_val["label"].append(curr_label)

In [10]:
df_train = pd.DataFrame(
    dict_train
)

df_val = pd.DataFrame(
    dict_val
)

In [11]:
class CaltechDataset(Dataset):
  def __init__(self, df, transforms):
    self.df = df
    self.transforms = transforms
    self.idx2label = {idx: self.df[self.df.label_idx == idx].label.values[0] for idx in set(self.df.label_idx.values)}
    self.label2idx = {label: idx for idx, label in self.idx2label.items()}
  
  def __getitem__(self, idx):
    img_path = self.df.iloc[idx].path
    label_idx = self.df.iloc[idx].label_idx
    
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    aug = self.transforms(image=img)
    img = aug["image"]
    
    return img, int(label_idx)

  def __len__(self):
    return len(self.df)

In [12]:
transforms_train = A.Compose([
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p=0.3),
    A.Rotate(30, p=0.3),
    A.Resize(256, 256),
    A.Normalize(),
    ToTensorV2()
])

transforms_val = A.Compose([
    A.Resize(256, 256),
    A.Normalize(),
    ToTensorV2()
])

In [13]:
train_dataset = CaltechDataset(df_train, transforms_train)
val_dataset = CaltechDataset(df_val, transforms_val)

image_datasets = {
    "train": train_dataset,
    "val": val_dataset,
}

In [29]:
dataloaders = {
    data_type: DataLoader(
        image_datasets[data_type],
        batch_size=32,
        shuffle=True
        )
    for data_type in ["train", "val"]
}

dataset_sizes = {
    data_type: len(image_datasets[data_type]) for data_type in ['train', 'val']
    }

In [30]:
#scheduler отвечает за изменение lr со временем в процессе обучения
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tqdm(dataloaders[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [31]:
model = models.resnet18()

In [32]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 256)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = optim.AdamW(model.parameters())

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [33]:
model = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------


100%|██████████| 565/565 [03:59<00:00,  2.35it/s]


train Loss: 5.1577 Acc: 0.0668


100%|██████████| 184/184 [01:04<00:00,  2.84it/s]


val Loss: 4.8225 Acc: 0.0930

Epoch 1/9
----------


 24%|██▍       | 137/565 [00:40<02:07,  3.35it/s]


KeyboardInterrupt: 